<a href="https://colab.research.google.com/github/KorStats/classification-project/blob/main/%EC%A6%9D%EB%B6%84/Q%EB%B6%84%EB%A5%98_%EC%A6%9D%EB%B6%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 구글드라이브에서 파일 읽어오기

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
 산업분류자동화		'# df_P_inc.csv'   images.zip	     Q_ori_model.pt
 classes.txt		 F_ori_model.pt    labels_1650.zip   S_ori_model.pt
'Colab Notebooks'	 H_ori_model.pt    labels.zip	     val.txt
'답안 작성용 파일.csv'	 images_1650.zip  'My Drive'


In [3]:
!unzip /content/gdrive/'My Drive'/산업분류자동화/'실습용 자료'.zip -d /content/dataset

Archive:  /content/gdrive/My Drive/산업분류자동화/실습용 자료.zip
1. 실습용자료.txt:  mismatching "local" filename (1. ьЛдьК╡ьЪйьЮРыгМ.txt),
         continuing with "central" filename version
  inflating: /content/dataset/1. 실습용자료.txt  


In [4]:
import pandas as pd
df=pd.read_csv('/content/dataset/1. 실습용자료.txt', sep = "|", encoding = "euc-kr")

## 데이터 전처리

In [5]:
df=df[(df['digit_1'] =='Q')] #조건에 맞는 데이터 출력

In [6]:
df=df.reset_index(drop=True)
df

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
0,id_0000005,Q,87,872,어린이집,보호자의 위탁을 받아,취학전아동보육
1,id_0000072,Q,86,863,보건소,내원 환자를 대상으로,진료서비스
2,id_0000101,Q,86,862,일반의원에서,외래환자를 위주로,외과진료
3,id_0000128,Q,87,872,복지관에서,일반인들을 대상으로,비거주복지서비스업
4,id_0000254,Q,87,872,어린이집에서,보육료,취학전 아동보육
...,...,...,...,...,...,...,...
36082,id_0999859,Q,87,872,복지관에서,일반인 대상으로,사회복지 서비스
36083,id_0999871,Q,87,872,어린이집,보호자의 위탁을 받아,취학전 아동보육
36084,id_0999916,Q,87,872,NaN,재가 요양서비스 기관,NaN
36085,id_0999960,Q,87,872,어린이집에서,보호자로 위탁받아,영유아보육


In [7]:
df['text']=df['text_obj'].map(str)+" "+df['text_mthd'].map(str)+" "+df['text_deal'].map(str)
i=[]
for sentence in df['text']:
    a=sentence.replace('nan', '')
    i.append(a)
df['text']=pd.DataFrame(i)
df=df.drop(['text_obj','text_mthd', 'text_deal'], axis=1) # 기존에 있던 세개의 text변수 삭제
df

,AI_id,digit_1,digit_2,digit_3,text
0,id_0000005,Q,87,872,어린이집 보호자의 위탁을 받아 취학전아동보육
1,id_0000072,Q,86,863,보건소 내원 환자를 대상으로 진료서비스
2,id_0000101,Q,86,862,일반의원에서 외래환자를 위주로 외과진료
3,id_0000128,Q,87,872,복지관에서 일반인들을 대상으로 비거주복지서비스업
4,id_0000254,Q,87,872,어린이집에서 보육료 취학전 아동보육
...,...,...,...,...,...
36082,id_0999859,Q,87,872,복지관에서 일반인 대상으로 사회복지 서비스
36083,id_0999871,Q,87,872,어린이집 보호자의 위탁을 받아 취학전 아동보육
36084,id_0999916,Q,87,872,재가 요양서비스 기관
36085,id_0999960,Q,87,872,어린이집에서 보호자로 위탁받아 영유아보육


In [8]:
df['digit_3'].value_counts()

872    15743
862    15287
871     2128
869     1198
861      878
863      853
Name: digit_3, dtype: int64

## **# 데이터 증분 및 맞춤법 교정**

In [ ]:
# 맞춤법 검사

In [9]:
!pip install git+https://github.com/ssut/py-hanspell

  Cloning https://github.com/ssut/py-hanspell to /tmp/pip-req-build-j6soacvr
  Running command git clone -q https://github.com/ssut/py-hanspell /tmp/pip-req-build-j6soacvr
  Created wheel for py-hanspell: filename=py_hanspell-1.1-py3-none-any.whl size=4868 sha256=3146f9178fb7f8a5e9150fe9834785df68f2b8b76a49630d4ed0bf704ea7a88b
  Stored in directory: /tmp/pip-ephem-wheel-cache-5cy0l795/wheels/31/c2/c7/ddbbd9a84ddb0a67c2e52e3324d4b7b47dcf9581af9cd784de
Successfully built py-hanspell


In [10]:
from hanspell import spell_checker

temp_result=[]
for sentence in df['text']:
  try:
    result=spell_checker.check(sentence)
    temp_result.append(result.checked)
  except:
    temp_result.append(sentence)

df['text']=pd.DataFrame(temp_result)  

In [11]:
df.to_csv('df_Q_spellcheck.csv', index=False, encoding='utf-8-sig')

In [12]:
df['digit_3'].value_counts()

872    15743
862    15287
871     2128
869     1198
861      878
863      853
Name: digit_3, dtype: int64

In [13]:
# 데이터 증분 코드
import random
import pickle
import re
def get_only_hangul(line):
	parseText= re.compile('/ ^[ㄱ-ㅎㅏ-ㅣ가-힣]*$/').sub('',line)
	return parseText

def synonym_replacement(words, n):
	new_words = words.copy()
	random_word_list = list(set([word for word in words]))
	random.shuffle(random_word_list)
	num_replaced = 0
	for random_word in random_word_list:
		synonyms = get_synonyms(random_word)
		if len(synonyms) >= 1:
			synonym = random.choice(list(synonyms))
			new_words = [synonym if word == random_word else word for word in new_words]
			num_replaced += 1
		if num_replaced >= n:
			break
	if len(new_words) != 0:
		sentence = ' '.join(new_words)
		new_words = sentence.split(" ")
	else:
		new_words = ""
	return new_words

def get_synonyms(word):
	synomyms = []
	try:
		for syn in wordnet[word]:
			for s in syn:
				synomyms.append(s)
	except:
		pass
	return synomyms

def random_deletion(words, p):
	if len(words) == 1:
		return words

	new_words = []
	for word in words:
		r = random.uniform(0, 1)
		if r > p:
			new_words.append(word)
	if len(new_words) == 0:
		rand_int = random.randint(0, len(words)-1)
		return [words[rand_int]]
	return new_words

########################################################################
# Random swap
# Randomly swap two words in the sentence n times
########################################################################
def random_swap(words, n):
	new_words = words.copy()
	for _ in range(n):
		new_words = swap_word(new_words)
	return new_words

def swap_word(new_words):
	random_idx_1 = random.randint(0, len(new_words)-1)
	random_idx_2 = random_idx_1
	counter = 0
	while random_idx_2 == random_idx_1:
		random_idx_2 = random.randint(0, len(new_words)-1)
		counter += 1
		if counter > 3:
			return new_words
	new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1]
	return new_words

########################################################################
# Random insertion
# Randomly insert n words into the sentence
########################################################################
def random_insertion(words, n):
	new_words = words.copy()
	for _ in range(n):
		add_word(new_words)	
	return new_words

def add_word(new_words):
	synonyms = []
	counter = 0
	while len(synonyms) < 1:
		if len(new_words) >= 1:
			random_word = new_words[random.randint(0, len(new_words)-1)]
			synonyms = get_synonyms(random_word)
			counter += 1
		else:
			random_word = ""
		if counter >= 10:
			return
		
	random_synonym = synonyms[0]
	random_idx = random.randint(0, len(new_words)-1)
	new_words.insert(random_idx, random_synonym)

def EDA(sentence, alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.1, p_rd=0.1, num_aug=9):
	sentence = get_only_hangul(sentence)
	words = sentence.split(' ')
	words = [word for word in words if word is not ""]
	num_words = len(words)
	augmented_sentences = []
	num_new_per_technique = int(num_aug/4) + 1
	n_sr = max(1, int(alpha_sr*num_words))
	n_ri = max(1, int(alpha_ri*num_words))
	n_rs = max(1, int(alpha_rs*num_words))
	# sr
	for _ in range(num_new_per_technique):
		a_words = synonym_replacement(words, n_sr)
		augmented_sentences.append(' '.join(a_words))
	# ri
	for _ in range(num_new_per_technique):
		a_words = random_insertion(words, n_ri)
		augmented_sentences.append(' '.join(a_words))
	# rs
	for _ in range(num_new_per_technique):
		a_words = random_swap(words, n_rs)
		augmented_sentences.append(" ".join(a_words))
	# rd
	for _ in range(num_new_per_technique):
		a_words = random_deletion(words, p_rd)
		augmented_sentences.append(" ".join(a_words))
	augmented_sentences = [get_only_hangul(sentence) for sentence in augmented_sentences]
	random.shuffle(augmented_sentences)
	if num_aug >= 1:
		augmented_sentences = augmented_sentences[:num_aug]
	else:
		keep_prob = num_aug / len(augmented_sentences)
		augmented_sentences = [s for s in augmented_sentences if random.uniform(0, 1) < keep_prob]
	augmented_sentences.append(sentence)
	return augmented_sentences

In [14]:
df1=df[df['digit_3']==851] 
df2=df[df['digit_3']==852] 
df3=df[df['digit_3']==853] 
df4=df[df['digit_3']==854] 
df5=df[df['digit_3']==855] 
df6=df[df['digit_3']==856] 
df7=df[df['digit_3']==857] 

df50=pd.concat([df1, df2])
df150=pd.concat([df7, df3, df4])

df50=df50.reset_index(drop=True)
df150=df150.reset_index(drop=True)

In [15]:
temp_result=[]
n=0
id_list=[]
digit_1_list=[]
digit_2_list=[]
digit_3_list=[]

for sentence in df50['text']:
  result_list=EDA(sentence, alpha_sr=0.0, alpha_ri=0.0, alpha_rs=0.5, p_rd=0.5, num_aug=5)
  id_list=['p'+str(df50['AI_id'][n]), 'p'+str(df50['AI_id'][n]), 'p'+str(df50['AI_id'][n]), 'p'+str(df50['AI_id'][n]),'p'+str(df50['AI_id'][n]), 'p'+str(df50['AI_id'][n])]
  digit_1_list=[df50['digit_1'][n], df50['digit_1'][n], df50['digit_1'][n], df50['digit_1'][n],df50['digit_1'][n], df50['digit_1'][n]]
  digit_2_list=[df50['digit_2'][n], df50['digit_2'][n], df50['digit_2'][n], df50['digit_2'][n],df50['digit_2'][n], df50['digit_2'][n]]
  digit_3_list=[df50['digit_3'][n], df50['digit_3'][n], df50['digit_3'][n], df50['digit_3'][n],df50['digit_3'][n], df50['digit_3'][n]]
  data_to_insert={'AI_id' : id_list, 'digit_1' : digit_1_list, 'digit_2' : digit_2_list, 'digit_3': digit_3_list, 'text' : result_list}
  df50_to_insert = pd.DataFrame(data_to_insert)
  df50= df50.append(df50_to_insert, ignore_index=True)
  n=n+1

In [16]:
df50

,AI_id,digit_1,digit_2,digit_3,text


In [17]:
temp_result=[]
n=0
id_list=[]
digit_1_list=[]
digit_2_list=[]
digit_3_list=[]

for sentence in df150['text']:
  result_list=EDA(sentence, alpha_sr=0.0, alpha_ri=0.0, alpha_rs=0.5, p_rd=0.5, num_aug=16)
  id_list=['p'+str(df150['AI_id'][n]), 'p'+str(df150['AI_id'][n]), 'p'+str(df150['AI_id'][n]), 'p'+str(df150['AI_id'][n]),'p'+str(df150['AI_id'][n]), 'p'+str(df150['AI_id'][n]), 'p'+str(df150['AI_id'][n]), 'p'+str(df150['AI_id'][n]),'p'+str(df150['AI_id'][n]), 'p'+str(df150['AI_id'][n]), 'p'+str(df150['AI_id'][n]), 'p'+str(df150['AI_id'][n]),'p'+str(df150['AI_id'][n]), 'p'+str(df150['AI_id'][n]), 'p'+str(df150['AI_id'][n]), 'p'+str(df150['AI_id'][n]),'p'+str(df150['AI_id'][n])]
  digit_1_list=[df150['digit_1'][n], df150['digit_1'][n], df150['digit_1'][n], df150['digit_1'][n],df150['digit_1'][n], df150['digit_1'][n], df150['digit_1'][n], df150['digit_1'][n],df150['digit_1'][n], df150['digit_1'][n], df150['digit_1'][n], df150['digit_1'][n],df150['digit_1'][n], df150['digit_1'][n], df150['digit_1'][n], df150['digit_1'][n],df150['digit_1'][n]]
  digit_2_list=[df150['digit_2'][n], df150['digit_2'][n], df150['digit_2'][n], df150['digit_2'][n],df150['digit_2'][n], df150['digit_2'][n], df150['digit_2'][n], df150['digit_2'][n],df150['digit_2'][n], df150['digit_2'][n], df150['digit_2'][n], df150['digit_2'][n],df150['digit_2'][n], df150['digit_2'][n], df150['digit_2'][n], df150['digit_2'][n],df150['digit_2'][n]]
  digit_3_list=[df150['digit_3'][n], df150['digit_3'][n], df150['digit_3'][n], df150['digit_3'][n],df150['digit_3'][n], df150['digit_3'][n], df150['digit_3'][n], df150['digit_3'][n],df150['digit_3'][n], df150['digit_3'][n], df150['digit_3'][n], df150['digit_3'][n],df150['digit_3'][n], df150['digit_3'][n], df150['digit_3'][n], df150['digit_3'][n],df150['digit_3'][n]]
  data_to_insert={'AI_id' : id_list, 'digit_1' : digit_1_list, 'digit_2' : digit_2_list, 'digit_3': digit_3_list, 'text' : result_list}
  df150_to_insert = pd.DataFrame(data_to_insert)
  df150= df150.append(df150_to_insert, ignore_index=True)
  n=n+1

In [18]:
df150

,AI_id,digit_1,digit_2,digit_3,text


In [19]:
df50=df50.reset_index(drop=True)
df150=df150.reset_index(drop=True)
df3=df3.reset_index(drop=True)
df7=df7.reset_index(drop=True)

기존 데이터 합치기

In [20]:
df=pd.concat([df3, df7, df50, df150])

In [21]:
df['digit_3'].value_counts()

Series([], Name: digit_3, dtype: int64)

In [22]:
df

,AI_id,digit_1,digit_2,digit_3,text


In [23]:
df.to_csv('df_Q_inc.csv', index=False, encoding='utf-8-sig')